# RAG ror PDF written purely in Python

First install a library to read pdf-files

In [277]:
# !pip install pymupdf

# Build the vector store

First we build the vector store based on our main document. Here we downloaded some paper.

In [278]:
the_document_path = "./data/2205.13147v4.pdf" 

In [279]:
from py4ragTools.database import Database

In [280]:
# embedding_model = 'all-minilm'  # 384
embedding_model='nomic-embed-text'  # 768
db = Database(embedding_model=embedding_model)

constructor here


In [281]:
from py4ragTools.text_tools import TextHelper, CharacterTextSplitter
th = TextHelper()

# load the document
doc = th.load(the_document_path)

# split the text into chunks
splitter = CharacterTextSplitter(chunk_size=800,chunk_overlap=200)
chunks = splitter.split(doc)
len(chunks)

212

In [282]:
# now we build the 'vector store' converting each chunk into a vector
count = 0
for chunk in chunks:
    # Bei Texten der Länge 0 bleibt ollama stehen
    # Sehr kurze Texte sind unbrauchbar für's anschliessende Generieren
    if len(chunk) > 10:
        # print(count)
        # print(entry)
        db.add(chunk)
        count += 1
print(f'{count} entries in db')  

212 entries in db


# Retrieval

We have built our vector store let us retrieve a context for a user-query.

Set up a query regarding today's topics on the newspaper's front page.

In [296]:
#  observ: sentance 'it is often the case...' available in context, but not in response
USER_PROMPT_0 = """
Are there computational and statistical constraints?
"""

#  observ: context has complete sub points response only returns first
USER_PROMPT_1 = """ \
What does the Checklist say, if you are using existing assets?
"""

# audience prompt pattern
#  observ: response sounds well, though hard to say if it fully makes sense
USER_PROMPT_2 = """ \
What is Matryoshka Representation Learning? Explain to me like I’m 11 years old.
"""

# persona  pattern
#  observ: response sounds well, though hard to say if it fully makes sense
USER_PROMPT_3 = """ \
What is Matryoshka Representation Learning? Act as a oriental story teller.
"""

# response template pattern
#  observ: works well
USER_PROMPT_4 = """ \
What is Matryoshka Representation Learning?
Format your answer as follows:
#Question: [The question being asked by User]
## Summary 
[a one paragraph summary of the answer]
## Details 
[a detailed answer]
"""

# question refinement pattern
# observ: this, does not semm make much sense in this szenarion because the llm needs to know the whole document to suggest more precise questions
USER_PROMPT_5 = """ \
Is Matryoshka Representation usefull at all?
Suggest a better version of the question and ask me if I would like to use it instead
"""

user_query = USER_PROMPT_3


In [297]:
context = db.query_database(user_query, 0.5 )
len(context)


88

In [298]:
context

[(0.7258670687494336,
  'oshka Representation Learning (MRL) to induce flexibility in the learned\nrepresentation. MRL learns representations of varying capacities within the same high-dimensional\nvector through explicit optimization of O(log(d)) lower-dimensional vectors in a nested fashion,\nhence the name Matryoshka. MRL can be adapted to any existing representation pipeline and\nis easily extended to many standard tasks in computer vision and natural language processing.\nFigure 1 illustrates the core idea of Matryoshka Representation Learning (MRL) and the adaptive\ndeployment settings of the learned Matryoshka Representations.\nAdaptive Retrieval\nShortlisting\nRe-ranking\nAdaptive Classification\nTraining\nInference\n<latexit sha1_base64="eh9hk+peBkdsPY6v+r4rONmxYLY=">A\nB7nicbVBNSwMxEJ2tX7V+VT16CRbBU9kVoR6LXjxWsB/QLiWb'),
 (0.7117174144365266,
  ', and signif-\nicantly for lower ones. This demonstrates that training to learn Matryoshka Representations\nis feasible and extendab

# Augmentation
Now that we have built our "vector database" and retrieved our context, we do the inference part:

First, we augment the prompt using the context we retrieved.

In [299]:
from py4ragTools.prompt_tools import PromptTools

pt = PromptTools()

In [300]:
# test some other system templates

SYS_PROMPT_1 = """ \
Context:
{context}
Based on the context provided above, answer the following question. Do not use any external knowledge.
If the answer is not present in the context, respond with "I don't know."
If the context is empty or irrelevant to the question, respond with "No relevant information found in the context."
"""

SYS_PROMPT_2 = """ \
Context:
{context}
Based on the context provided above, answer the following question. Do not use any external knowledge.
Ensure your response cites the relevant sentence numbers from the context for each fact you provide."
"""

SYS_PROMPT_3 = """ \
Context:
{context}
Based on the context provided above, answer the question. Do only use information from the provided context in your answer"
"""


pt.SYSTEM_TEMPLATE = SYS_PROMPT_1

In [301]:
context_prompt = ""
for (score, text) in context:
    context_prompt += (text + "\\n")
print(context_prompt)

p_sys_ddic = pt.system_prompt(context_prompt)

oshka Representation Learning (MRL) to induce flexibility in the learned
representation. MRL learns representations of varying capacities within the same high-dimensional
vector through explicit optimization of O(log(d)) lower-dimensional vectors in a nested fashion,
hence the name Matryoshka. MRL can be adapted to any existing representation pipeline and
is easily extended to many standard tasks in computer vision and natural language processing.
Figure 1 illustrates the core idea of Matryoshka Representation Learning (MRL) and the adaptive
deployment settings of the learned Matryoshka Representations.
Adaptive Retrieval
Shortlisting
Re-ranking
Adaptive Classification
Training
Inference
<latexit sha1_base64="eh9hk+peBkdsPY6v+r4rONmxYLY=">A
B7nicbVBNSwMxEJ2tX7V+VT16CRbBU9kVoR6LXjxWsB/QLiWb\n, and signif-
icantly for lower ones. This demonstrates that training to learn Matryoshka Representations
is feasible and extendable even for extremely large scale datasets.
We also demonstrate that

In [302]:
# view the full sy prompt comntent
print(p_sys_ddic['content'])

 Context:
oshka Representation Learning (MRL) to induce flexibility in the learned
representation. MRL learns representations of varying capacities within the same high-dimensional
vector through explicit optimization of O(log(d)) lower-dimensional vectors in a nested fashion,
hence the name Matryoshka. MRL can be adapted to any existing representation pipeline and
is easily extended to many standard tasks in computer vision and natural language processing.
Figure 1 illustrates the core idea of Matryoshka Representation Learning (MRL) and the adaptive
deployment settings of the learned Matryoshka Representations.
Adaptive Retrieval
Shortlisting
Re-ranking
Adaptive Classification
Training
Inference
<latexit sha1_base64="eh9hk+peBkdsPY6v+r4rONmxYLY=">A
B7nicbVBNSwMxEJ2tX7V+VT16CRbBU9kVoR6LXjxWsB/QLiWb\n, and signif-
icantly for lower ones. This demonstrates that training to learn Matryoshka Representations
is feasible and extendable even for extremely large scale datasets.
We also demons

In [303]:
p_user_ddic = pt.user_prompt(user_query)

In [304]:
# view the full sy prompt comntent
print(p_user_ddic['content'])

 Query:  What is Matryoshka Representation Learning? Act as a oriental story teller.




In [305]:
# create list of system and user prompt dictionary
list_of_prompts = [
    pt.system_prompt(context_prompt),
    # pt.user_prompt(user_query + "\nProvide the context used for your explanations.")
    pt.user_prompt(user_query)
]

print(list_of_prompts)

[{'role': 'system', 'content': ' Context:\noshka Representation Learning (MRL) to induce flexibility in the learned\nrepresentation. MRL learns representations of varying capacities within the same high-dimensional\nvector through explicit optimization of O(log(d)) lower-dimensional vectors in a nested fashion,\nhence the name Matryoshka. MRL can be adapted to any existing representation pipeline and\nis easily extended to many standard tasks in computer vision and natural language processing.\nFigure 1 illustrates the core idea of Matryoshka Representation Learning (MRL) and the adaptive\ndeployment settings of the learned Matryoshka Representations.\nAdaptive Retrieval\nShortlisting\nRe-ranking\nAdaptive Classification\nTraining\nInference\n<latexit sha1_base64="eh9hk+peBkdsPY6v+r4rONmxYLY=">A\nB7nicbVBNSwMxEJ2tX7V+VT16CRbBU9kVoR6LXjxWsB/QLiWb\\n, and signif-\nicantly for lower ones. This demonstrates that training to learn Matryoshka Representations\nis feasible and extendable even 

# Generation

Let us generate a response using the augmented prompt.

First, initialize the generation model:

In [306]:
from py4ragTools.generation_tools import GenerationTools
#gt = GenerationTools(generation_model='llama3.1:8b')
gt = GenerationTools(generation_model='llama3.1')

constructor here


In [307]:
response = gt.get_response(list_of_prompts) 
response.choices[0].message.content

'Gather \'round, young scholars, and listen well to the tale of... (dramatic pause) ...Matryoshka Representation Learning!\n\n(In a thick, mystical voice)\n\nIn days of yore, when the world was young and mysterious, there lived a great philosopher-king named Mikhail. He sought to unlock the secrets of representation learning, inscribing on his scroll: "To learn what is beneath the surface, one must start with the obvious."\n\nAnd lo, he created a method, known as Matryoshka Representation Learning (MRL). It\'s a technique that weaves together multiple learning frameworks into one mighty thread.\n\nImagine a sturdy wooden doll – the Nest – containing a smaller, intricately carved version of itself inside (Daughter Doll). Within this Daughter Doll lies an even smaller replica (Granddaughter Doll), and so on. The Matryoshka family grows with each level, yet remains connected through shared knowledge and wisdom.\n\nIn this symbolic context, MRL represents learning various representations u

In [308]:
print(response.choices[0].message.content)

Gather 'round, young scholars, and listen well to the tale of... (dramatic pause) ...Matryoshka Representation Learning!

(In a thick, mystical voice)

In days of yore, when the world was young and mysterious, there lived a great philosopher-king named Mikhail. He sought to unlock the secrets of representation learning, inscribing on his scroll: "To learn what is beneath the surface, one must start with the obvious."

And lo, he created a method, known as Matryoshka Representation Learning (MRL). It's a technique that weaves together multiple learning frameworks into one mighty thread.

Imagine a sturdy wooden doll – the Nest – containing a smaller, intricately carved version of itself inside (Daughter Doll). Within this Daughter Doll lies an even smaller replica (Granddaughter Doll), and so on. The Matryoshka family grows with each level, yet remains connected through shared knowledge and wisdom.

In this symbolic context, MRL represents learning various representations using differen